# T1055.001 Dynamic-link Library Injection

-----------------------------------------------------------------------

## Technique Description

Adversaries may inject dynamic-link libraries (DLLs) into processes in order to evade process-based defenses as well as possibly elevate privileges. DLL injection is a method of executing arbitrary code in the address space of a separate live process.  

DLL injection is commonly performed by writing the path to a DLL in the virtual address space of the target process before loading the DLL by invoking a new thread. The write can be performed with native Windows API calls such as <code>VirtualAllocEx</code> and <code>WriteProcessMemory</code>, then invoked with <code>CreateRemoteThread</code> (which calls the <code>LoadLibrary</code> API responsible for loading the DLL). (Citation: Elastic Process Injection July 2017) 

Variations of this method such as reflective DLL injection (writing a self-mapping DLL into a process) and memory module (map DLL when writing into process) overcome the address relocation issue as well as the additional APIs to invoke execution (since these methods load and execute the files in memory by manually preforming the function of <code>LoadLibrary</code>).(Citation: Elastic HuntingNMemory June 2017)(Citation: Elastic Process Injection July 2017) 

Running code in the context of another process may allow access to the process's memory, system/network resources, and possibly elevated privileges. Execution via DLL injection may also evade detection from security products since the execution is masked under a legitimate process. 

## Technique Detection

Monitoring Windows API calls indicative of the various types of code injection may generate a significant amount of data and may not be directly useful for defense unless collected under specific circumstances for known bad sequences of calls, since benign use of API functions may be common and difficult to distinguish from malicious behavior. Windows API calls such as <code>CreateRemoteThread</code> and those that can be used to modify memory within another process, such as <code>VirtualAllocEx</code>/<code>WriteProcessMemory</code>, may be used for this technique.(Citation: Elastic Process Injection July 2017)

Monitor DLL/PE file events, specifically creation of these binary files as well as the loading of DLLs into processes. Look for DLLs that are not recognized or not normally loaded into a process. 

Analyze process behavior to determine if a process is performing actions it usually does not, such as opening network connections, reading files, or other suspicious actions that could relate to post-compromise behavior. 

-----------------------------------------------------------------------

### Tactics:

  * Defense-Evasion

  * Privilege-Escalation

### Platforms:

  * Windows

### Adversary Required Permissions:

  * User

### Defenses Bypassed:

  * Application control

  * Anti-virus

### Data Sources:

  * **Process:** Process Access

  * **Process:** OS API Execution

  * **Module:** Module Load

  * **Process:** Process Modification

-----------------------------------------------------------------------

### Adversarial usage:

| Adversary Group |  Adversarial Usage |
|----|----|
| BackdoorDiplomacy | [BackdoorDiplomacy](https://attack.mitre.org/groups/G0135) has dropped legitimate software onto a compromised host and used it to execute malicious DLLs.(Citation: ESET BackdoorDiplomacy Jun 2021)| 
| Wizard Spider | [Wizard Spider](https://attack.mitre.org/groups/G0102) has injected malicious DLLs into memory with read, write, and execute permissions.(Citation: DHS/CISA Ransomware Targeting Healthcare October 2020)(Citation: DFIR Ryuk 2 Hour Speed Run November 2020)| 
| TA505 | [TA505](https://attack.mitre.org/groups/G0092) has been seen injecting a DLL into winword.exe.(Citation: IBM TA505 April 2020)| 
| Tropic Trooper | [Tropic Trooper](https://attack.mitre.org/groups/G0081) has injected a DLL backdoor into dllhost.exe and svchost.exe.(Citation: TrendMicro Tropic Trooper Mar 2018)(Citation: TrendMicro Tropic Trooper May 2020)| 
| Leviathan | [Leviathan](https://attack.mitre.org/groups/G0065) has utilized techniques like reflective DLL loading to write a DLL into memory and load a shell that provides backdoor access to the victim.(Citation: Accenture MUDCARP March 2019)| 
| Lazarus Group | A [Lazarus Group](https://attack.mitre.org/groups/G0032) malware sample performs reflective DLL injection.(Citation: McAfee Lazarus Resurfaces Feb 2018)(Citation: Lazarus APT January 2022)| 
| Putter Panda | An executable dropped onto victims by [Putter Panda](https://attack.mitre.org/groups/G0024) aims to inject the specified DLL into a process that would normally be accessing the network, including Outlook Express (msinm.exe), Outlook (outlook.exe), Internet Explorer (iexplore.exe), and Firefox (firefox.exe).(Citation: CrowdStrike Putter Panda)| 
| Turla | [Turla](https://attack.mitre.org/groups/G0010) has used Metasploit to perform reflective DLL injection in order to escalate privileges.(Citation: ESET Turla Mosquito May 2018)(Citation: Github Rapid7 Meterpreter Elevate)| 
-----------------------------------------------------------------------

## Mitre References

  * [Mitre-Attack](https://attack.mitre.org/techniques/T1055/001)

  * [Elastic Process Injection July 2017](https://www.endgame.com/blog/technical-blog/ten-process-injection-techniques-technical-survey-common-and-trending-process), Hosseini, A. (2017, July 18). Ten Process Injection Techniques: A Technical Survey Of Common And Trending Process Injection Techniques. Retrieved December 7, 2017.

  * [Elastic Huntingnmemory June 2017](https://www.endgame.com/blog/technical-blog/hunting-memory), Desimone, J. (2017, June 13). Hunting in Memory. Retrieved December 7, 2017.

> *Note: Do not edit this cell with information you want to keep. This cell will be wiped when the update script is ran. Store permanent information in one of the relevant cells below*

*Last pulled from Mitre on: 23 June 2022*



-----------------------------------------------------------------------

## Metadata

  * **Last Updated  Date:** 8 July 2021

  * **Author(s):** SSgt Johnathan Smith, SSgt John Beres

  * **Validated:** NO

-----------------------------------------------------------------------

## Overall Hypothesis

- Adversary may use Metasploit to perform reflective DLL injection in order to escalate privileges.

## Adversary Examples

| Adversary Specific Examples | Host Analytics | Network Analytics |
|-----------------------------|----------------|-------------------|
| Example APT/Threat actor | 1, 2 | 2, 3, 5|

## Detection Blindspots

- Information Here

## Analytical References

  * [Turla Mosquito Shift Towards Generic Tools 2018 (welivesecurity)](https://www.welivesecurity.com/2018/05/22/turla-mosquito-shift-towards-generic-tools/)
  * [Hunting Memory (elastic)](https://www.elastic.co/blog/hunting-memory)

-----------------------------------------------------------------------

## Host Analytics

-----------------------------------------------------------------------

### Hunter Notes

- Work with MP to determine normal process behaviors to identify suspicious activity.
- Large RWX memory sections in a process is a good indicator that classic reflective DLL injection is being used.

#### Analytic 1

  * **Information:** Monitor DLL/PE file events, specifically creation of these binary files as well as the loading of DLLs into processes.

  * **Source:** Windows Audits, Sysmon

  * **Tool:** Kibana

  * **Notes:** Look for DLLs that are not recognized or not normally loaded into a process.

  * **Query:** ```event_id:7```

#### Analytic 2

  * **Information:** Analyze process behavior to determine if a process is performing actions it usually does not, such as opening network connections, reading files, or other suspicious actions that could relate to post-compromise behavior.

  * **Source:** Windows Audits, Sysmon

  * **Tool:** Kibana

  * **Notes:** 

  * **Query:** ```event_id:3 OR event_id:560```



-----------------------------------------------------------------------

## Network Analytics

-----------------------------------------------------------------------

### Hunter Notes

- Information Here

#### Analytic 1

  * **Information:** 'Comments'

  * **Source:** 'ex Windows Audits, Sysmon'

  * **Tool:** 'Arkime, Kibana, Autopsy'

  * **Notes:** 'Various notes to assist with hunting.'

  * **Query:** ```Input your query here```

#### Analytic 2

  * **Information:** 'Comments'

  * **Source:** 'ex Windows Audits, Sysmon'

  * **Tool:** 'Arkime, Kibana, Autopsy'

  * **Notes:** 'Various notes to assist with hunting.'

  * **Query:** ```Input your query here```

